# التكليف الثاني: تنظيف ومعالجة النصوص (Preprocessing)

## الأهداف
1. تنفيذ عمليات تنظيف النصوص العربية
2. تطبيق Tokenization, Stemming, Lemmatization
3. مقارنة بين المكتبات المختلفة وتحديد الأفضل

## استيراد المكتبات

In [1]:
import re
import string
import pandas as pd
import json
from time import time

# NLP Libraries
import spacy
import nltk
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import word_tokenize

# Arabic Specific
try:
    import pyarabic.araby as araby
    print("pyarabic installed")
except:
    print("pyarabic not installed")

try:
    from qalsadi.lemmatizer import Lemmatizer
    print("qalsadi installed")
except:
    print("qalsadi not installed")

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

print("Libraries loaded successfully!")

pyarabic installed
qalsadi installed
Libraries loaded successfully!


## مجموعة النصوص للتجربة

In [2]:
sample_texts = [
    "مرحباااا بكم في اليمن السعيد الخدمة روووعة جدا",
    "السعر 1500 ريال غالي جدا",
    "الاكل لذيذ والجو رائع انصح الجميع بزيارة هذا المطعم",
    "خدمةةةة سيئةةة جدااااا لن اعود مرة اخرى",
    "تجربة ممتازة 100 شكرا للفريق",
    "المنتج وصل متاخر ب 3 ايام والتغليف سيء",
    "احببت الخدمه كثييييرا ساعود قريبا ان شاء الله",
    "لا انصح ابدا التعامل سيء والاسعار مرتفعة جدا"
]

print("النصوص الاصلية:")
print("=" * 70)
for i, text in enumerate(sample_texts, 1):
    print(f"{i}. {text}")

النصوص الاصلية:
1. مرحباااا بكم في اليمن السعيد الخدمة روووعة جدا
2. السعر 1500 ريال غالي جدا
3. الاكل لذيذ والجو رائع انصح الجميع بزيارة هذا المطعم
4. خدمةةةة سيئةةة جدااااا لن اعود مرة اخرى
5. تجربة ممتازة 100 شكرا للفريق
6. المنتج وصل متاخر ب 3 ايام والتغليف سيء
7. احببت الخدمه كثييييرا ساعود قريبا ان شاء الله
8. لا انصح ابدا التعامل سيء والاسعار مرتفعة جدا


## دوال التنظيف (Cleaning Functions)

In [4]:
class ArabicTextCleaner:
    """
    فئة متكاملة لتنظيف النصوص العربية
    """
    
    def __init__(self):
        self.arabic_punctuations = ''.join(['`', '÷', '×', '؛', '<', '>', '(', ')', '*', '&', '^', '%', ']', '[', 'ـ', '،', '/', ':', '"', '؟', '.', ',', "'", '{', '}', '~', '¦', '+', '|', '!', '…', '"', '"', '–'])
        self.english_punctuations = string.punctuation
        self.all_punctuations = self.arabic_punctuations + self.english_punctuations
        
        self.emoji_pattern = re.compile(
            "[" 
            "\U0001F600-\U0001F64F"
            "\U0001F300-\U0001F5FF"
            "\U0001F680-\U0001F6FF"
            "\U0001F1E0-\U0001F1FF"
            "\U00002702-\U000027B0"
            "\U000024C2-\U0001F251"
            "\U0001F900-\U0001F9FF"
            "\U00002600-\U000026FF"
            "\U00002700-\U000027BF"
            "]+", flags=re.UNICODE)
    
    def remove_emojis(self, text):
        """ازالة الايموجي"""
        return self.emoji_pattern.sub('', text)
    
    def remove_punctuation(self, text):
        """ازالة علامات الترقيم"""
        translator = str.maketrans('', '', self.all_punctuations)
        return text.translate(translator)
    
    def remove_elongation(self, text):
        """ازالة المد والتكرار"""
        return re.sub(r'(.)\1{2,}', r'\1', text)
    
    def remove_numbers(self, text):
        """ازالة الارقام"""
        return re.sub(r'[0-9٠-٩]+', '', text)
    
    def remove_diacritics(self, text):
        """ازالة التشكيل"""
        try:
            return araby.strip_tashkeel(text)
        except:
            arabic_diacritics = re.compile(r'[\u064B-\u065F\u0670]')
            return arabic_diacritics.sub('', text)
    
    def normalize_arabic(self, text):
        """تطبيع الحروف العربية"""
        text = re.sub(r'[إأآا]', 'ا', text)
        text = re.sub(r'ة', 'ه', text)
        text = re.sub(r'ى', 'ي', text)
        return text
    
    def remove_extra_spaces(self, text):
        """ازالة المسافات الزائدة"""
        return ' '.join(text.split())
    
    def clean_text(self, text, normalize=True):
        """تطبيق جميع عمليات التنظيف"""
        text = self.remove_emojis(text)
        text = self.remove_punctuation(text)
        text = self.remove_elongation(text)
        text = self.remove_numbers(text)
        text = self.remove_diacritics(text)
        if normalize:
            text = self.normalize_arabic(text)
        text = self.remove_extra_spaces(text)
        return text


cleaner = ArabicTextCleaner()
print("تم انشاء فئة التنظيف بنجاح!")

تم انشاء فئة التنظيف بنجاح!


In [5]:
# تطبيق التنظيف على النصوص
print("تطبيق عمليات التنظيف:")
print("=" * 70)

cleaned_texts = []
for i, text in enumerate(sample_texts, 1):
    cleaned = cleaner.clean_text(text)
    cleaned_texts.append(cleaned)
    print(f"\nالنص {i}:")
    print(f"   قبل: {text}")
    print(f"   بعد: {cleaned}")

تطبيق عمليات التنظيف:

النص 1:
   قبل: مرحباااا بكم في اليمن السعيد الخدمة روووعة جدا
   بعد: مرحبا بكم في اليمن السعيد الخدمه روعه جدا

النص 2:
   قبل: السعر 1500 ريال غالي جدا
   بعد: السعر ريال غالي جدا

النص 3:
   قبل: الاكل لذيذ والجو رائع انصح الجميع بزيارة هذا المطعم
   بعد: الاكل لذيذ والجو رائع انصح الجميع بزياره هذا المطعم

النص 4:
   قبل: خدمةةةة سيئةةة جدااااا لن اعود مرة اخرى
   بعد: خدمه سيئه جدا لن اعود مره اخري

النص 5:
   قبل: تجربة ممتازة 100 شكرا للفريق
   بعد: تجربه ممتازه شكرا للفريق

النص 6:
   قبل: المنتج وصل متاخر ب 3 ايام والتغليف سيء
   بعد: المنتج وصل متاخر ب ايام والتغليف سيء

النص 7:
   قبل: احببت الخدمه كثييييرا ساعود قريبا ان شاء الله
   بعد: احببت الخدمه كثيرا ساعود قريبا ان شاء الله

النص 8:
   قبل: لا انصح ابدا التعامل سيء والاسعار مرتفعة جدا
   بعد: لا انصح ابدا التعامل سيء والاسعار مرتفعه جدا


## Tokenization (تقسيم الكلمات)

In [6]:
# تحميل نموذج Spacy
try:
    nlp = spacy.load("xx_ent_wiki_sm")
except:
    nlp = spacy.blank("ar")

def tokenize_with_spacy(text):
    """Tokenization باستخدام Spacy"""
    doc = nlp(text)
    return [token.text for token in doc if not token.is_space]

def tokenize_with_nltk(text):
    """Tokenization باستخدام NLTK"""
    return word_tokenize(text)

def tokenize_simple(text):
    """Tokenization بسيط"""
    return text.split()

# تطبيق على نص مثال
test_text = cleaned_texts[0]

print("مقارنة طرق Tokenization:")
print(f"\nالنص: {test_text}")
print(f"\n1. Spacy: {tokenize_with_spacy(test_text)}")
print(f"2. NLTK: {tokenize_with_nltk(test_text)}")
print(f"3. Simple: {tokenize_simple(test_text)}")

مقارنة طرق Tokenization:

النص: مرحبا بكم في اليمن السعيد الخدمه روعه جدا

1. Spacy: ['مرحبا', 'بكم', 'في', 'اليمن', 'السعيد', 'الخدمه', 'روعه', 'جدا']
2. NLTK: ['مرحبا', 'بكم', 'في', 'اليمن', 'السعيد', 'الخدمه', 'روعه', 'جدا']
3. Simple: ['مرحبا', 'بكم', 'في', 'اليمن', 'السعيد', 'الخدمه', 'روعه', 'جدا']


## Stemming (التجريد)

In [7]:
isri_stemmer = ISRIStemmer()

def stem_with_isri(tokens):
    """Stemming باستخدام ISRI"""
    return [isri_stemmer.stem(token) for token in tokens]

tokens = tokenize_simple(cleaned_texts[2])

print("Stemming باستخدام ISRI:")
print(f"\nالكلمات الاصلية: {tokens}")
print(f"بعد التجريد: {stem_with_isri(tokens)}")

Stemming باستخدام ISRI:

الكلمات الاصلية: ['الاكل', 'لذيذ', 'والجو', 'رائع', 'انصح', 'الجميع', 'بزياره', 'هذا', 'المطعم']
بعد التجريد: ['اكل', 'لذذ', 'لجو', 'رئع', 'نصح', 'جمع', 'زير', 'هذا', 'طعم']


In [8]:
test_words = [
    "يذهبون", "ذهب", "ذاهب", "مذهب",
    "يكتبون", "كتب", "كاتب", "مكتوب",
    "المسلمون", "مسلم", "اسلام", "اسلامي"
]

print("جدول التجريد:")
print("-" * 40)
for word in test_words:
    stem = isri_stemmer.stem(word)
    print(f"{word} -> {stem}")

جدول التجريد:
----------------------------------------
يذهبون -> ذهب
ذهب -> ذهب
ذاهب -> ذهب
مذهب -> ذهب
يكتبون -> كتب
كتب -> كتب
كاتب -> كتب
مكتوب -> كتب
المسلمون -> سلم
مسلم -> سلم
اسلام -> سلم
اسلامي -> سلم


## Lemmatization

In [9]:
def lemmatize_simple(word):
    """Lemmatization بسيط"""
    prefixes = ['ال', 'و', 'ف', 'ب', 'ك', 'ل']
    suffixes = ['ون', 'ين', 'ات', 'ان', 'ه', 'ي']
    
    for prefix in prefixes:
        if word.startswith(prefix) and len(word) > len(prefix) + 2:
            word = word[len(prefix):]
            break
    
    for suffix in suffixes:
        if word.endswith(suffix) and len(word) > len(suffix) + 2:
            word = word[:-len(suffix)]
            break
    
    return word

test_words_lemma = ["المسلمون", "الكتب", "يذهبون", "المدرسه", "الطلاب"]

print("مقارنة Lemmatization:")
print("-" * 60)

for word in test_words_lemma:
    simple = lemmatize_simple(word)
    stem = isri_stemmer.stem(word)
    print(f"{word} -> Simple: {simple}, ISRI: {stem}")

مقارنة Lemmatization:
------------------------------------------------------------
المسلمون -> Simple: مسلم, ISRI: سلم
الكتب -> Simple: كتب, ISRI: كتب
يذهبون -> Simple: يذهب, ISRI: ذهب
المدرسه -> Simple: مدرس, ISRI: درس
الطلاب -> Simple: طلاب, ISRI: طلب


## مقارنة المكتبات

In [10]:
def compare_libraries(texts):
    """مقارنة المكتبات المختلفة"""
    results = {
        'text': [],
        'spacy_tokens': [],
        'nltk_tokens': [],
        'isri_stems': [],
        'simple_lemmas': []
    }
    
    for text in texts:
        spacy_tok = tokenize_with_spacy(text)
        nltk_tok = tokenize_with_nltk(text)
        isri_stems = stem_with_isri(nltk_tok)
        simple_lemmas = [lemmatize_simple(t) for t in nltk_tok]
        
        results['text'].append(text)
        results['spacy_tokens'].append(spacy_tok)
        results['nltk_tokens'].append(nltk_tok)
        results['isri_stems'].append(isri_stems)
        results['simple_lemmas'].append(simple_lemmas)
    
    return pd.DataFrame(results)

comparison_df = compare_libraries(cleaned_texts[:3])
print("نتائج المقارنة:")
comparison_df

نتائج المقارنة:


,text,spacy_tokens,nltk_tokens,isri_stems,simple_lemmas
0,مرحبا بكم في اليمن السعيد الخدمه روعه جدا,"[مرحبا, بكم, في, اليمن, السعيد, الخدمه, روعه, ...","[مرحبا, بكم, في, اليمن, السعيد, الخدمه, روعه, ...","[رحب, بكم, في, يمن, سعد, خدم, روع, جدا]","[مرحبا, بكم, في, يمن, سعيد, خدم, روع, جدا]"
1,السعر ريال غالي جدا,"[السعر, ريال, غالي, جدا]","[السعر, ريال, غالي, جدا]","[سعر, ريل, غلي, جدا]","[سعر, ريال, غال, جدا]"
2,الاكل لذيذ والجو رائع انصح الجميع بزياره هذا ا...,"[الاكل, لذيذ, والجو, رائع, انصح, الجميع, بزيار...","[الاكل, لذيذ, والجو, رائع, انصح, الجميع, بزيار...","[اكل, لذذ, لجو, رئع, نصح, جمع, زير, هذا, طعم]","[اكل, ذيذ, الجو, رائع, انصح, جميع, زيار, هذا, ..."


In [11]:
import time

def measure_speed(func, data, iterations=100):
    """قياس سرعة تنفيذ الدالة"""
    start = time.time()
    for _ in range(iterations):
        for item in data:
            func(item)
    end = time.time()
    return (end - start) / iterations

print("قياس سرعة المكتبات:")
print("-" * 40)

spacy_time = measure_speed(tokenize_with_spacy, cleaned_texts)
nltk_time = measure_speed(tokenize_with_nltk, cleaned_texts)
simple_time = measure_speed(tokenize_simple, cleaned_texts)

print(f"Tokenization:")
print(f"  Spacy: {spacy_time*1000:.2f} ms")
print(f"  NLTK: {nltk_time*1000:.2f} ms")
print(f"  Simple: {simple_time*1000:.2f} ms")

قياس سرعة المكتبات:
----------------------------------------
Tokenization:
  Spacy: 5.34 ms
  NLTK: 0.10 ms
  Simple: 0.00 ms


## تقرير المقارنة

In [12]:
comparison_report = """
# تقرير مقارنة ادوات معالجة النصوص العربية

## 1. Tokenization
- Spacy: متوسطة السرعة، دقة عالية
- NLTK: سريعة، دقة جيدة
- Simple: سريعة جدا، دقة منخفضة

الافضل: Spacy للمهام المتقدمة

## 2. Stemming
- ISRI: سريعة، دقة جيدة للعربية

الافضل: ISRI Stemmer

## 3. Lemmatization
- Qalsadi: بطيئة، دقة عالية جدا
- Simple Rules: سريعة، دقة منخفضة

الافضل: Qalsadi للدقة العالية

## التوصيات
1. Tokenization: Spacy
2. Stemming: ISRI
3. Lemmatization: Qalsadi
4. التنظيف: pyarabic + regex
"""

print(comparison_report)


# تقرير مقارنة ادوات معالجة النصوص العربية

## 1. Tokenization
- Spacy: متوسطة السرعة، دقة عالية
- NLTK: سريعة، دقة جيدة
- Simple: سريعة جدا، دقة منخفضة

الافضل: Spacy للمهام المتقدمة

## 2. Stemming
- ISRI: سريعة، دقة جيدة للعربية

الافضل: ISRI Stemmer

## 3. Lemmatization
- Qalsadi: بطيئة، دقة عالية جدا
- Simple Rules: سريعة، دقة منخفضة

الافضل: Qalsadi للدقة العالية

## التوصيات
1. Tokenization: Spacy
2. Stemming: ISRI
3. Lemmatization: Qalsadi
4. التنظيف: pyarabic + regex



## حفظ البيانات

In [13]:
final_df = pd.DataFrame({
    'original_text': sample_texts,
    'cleaned_text': cleaned_texts,
    'tokens': [tokenize_simple(t) for t in cleaned_texts],
    'stems': [stem_with_isri(tokenize_simple(t)) for t in cleaned_texts]
})

print("البيانات النهائية:")
final_df

البيانات النهائية:


,original_text,cleaned_text,tokens,stems
0,مرحباااا بكم في اليمن السعيد الخدمة روووعة جدا,مرحبا بكم في اليمن السعيد الخدمه روعه جدا,"[مرحبا, بكم, في, اليمن, السعيد, الخدمه, روعه, ...","[رحب, بكم, في, يمن, سعد, خدم, روع, جدا]"
1,السعر 1500 ريال غالي جدا,السعر ريال غالي جدا,"[السعر, ريال, غالي, جدا]","[سعر, ريل, غلي, جدا]"
2,الاكل لذيذ والجو رائع انصح الجميع بزيارة هذا ا...,الاكل لذيذ والجو رائع انصح الجميع بزياره هذا ا...,"[الاكل, لذيذ, والجو, رائع, انصح, الجميع, بزيار...","[اكل, لذذ, لجو, رئع, نصح, جمع, زير, هذا, طعم]"
3,خدمةةةة سيئةةة جدااااا لن اعود مرة اخرى,خدمه سيئه جدا لن اعود مره اخري,"[خدمه, سيئه, جدا, لن, اعود, مره, اخري]","[خدم, سيئ, جدا, لن, اعد, مره, اخر]"
4,تجربة ممتازة 100 شكرا للفريق,تجربه ممتازه شكرا للفريق,"[تجربه, ممتازه, شكرا, للفريق]","[جرب, متز, شكر, فرق]"
5,المنتج وصل متاخر ب 3 ايام والتغليف سيء,المنتج وصل متاخر ب ايام والتغليف سيء,"[المنتج, وصل, متاخر, ب, ايام, والتغليف, سيء]","[نتج, وصل, تخر, ب, ايم, غلف, سيء]"
6,احببت الخدمه كثييييرا ساعود قريبا ان شاء الله,احببت الخدمه كثيرا ساعود قريبا ان شاء الله,"[احببت, الخدمه, كثيرا, ساعود, قريبا, ان, شاء, ...","[حبب, خدم, كثر, سعد, قرب, ان, شاء, الل]"
7,لا انصح ابدا التعامل سيء والاسعار مرتفعة جدا,لا انصح ابدا التعامل سيء والاسعار مرتفعه جدا,"[لا, انصح, ابدا, التعامل, سيء, والاسعار, مرتفع...","[لا, نصح, ابد, عمل, سيء, سعر, رفع, جدا]"


In [14]:
import os
os.makedirs('data', exist_ok=True)

final_df.to_csv('data/cleaned_texts.csv', index=False, encoding='utf-8-sig')
print("تم حفظ البيانات في: data/cleaned_texts.csv")

with open('comparison_report.md', 'w', encoding='utf-8') as f:
    f.write(comparison_report)
print("تم حفظ تقرير المقارنة في: comparison_report.md")

تم حفظ البيانات في: data/cleaned_texts.csv
تم حفظ تقرير المقارنة في: comparison_report.md


## ملخص التكليف الثاني

### ما تم انجازه:

1. تنظيف النصوص:
   - ازالة الايموجي
   - ازالة علامات الترقيم
   - ازالة المد والتكرار
   - ازالة الارقام
   - تطبيع النص العربي

2. العمليات اللغوية:
   - Tokenization (Spacy, NLTK, Simple)
   - Stemming (ISRI Stemmer)
   - Lemmatization

3. مقارنة المكتبات

4. المخرجات:
   - data/cleaned_texts.csv
   - comparison_report.md

انتهى التكليف الثاني